## Jonas ideas for a notebook

##### Setup environment

In [ ]:
import sys
sys.path.append("..")
from psflearning.psflearninglib import psflearninglib
from psflearning import io
from psflearning.makeplots import *

##### **Set necessary paprameters**

In [ ]:
PSFtype = 'zernike_vector' # PSF model class: voxel, zernike, zernike_vector, pupil, pupil_vector
Datapath = 'D:/DataSSD/testdata/BeadsM2/' # Path to data
Keyword = 'Pos'   # keyword for your data file, used for automatic finding files with the same keyword
Microscope_config = 'M2'
Spline_focus_depth = 0.5 # micro meter, for spline PSF calculation. Only zernike / pupil model. Or make separate conversion notebook

##### set default parameters, do some calculations, hide later, if possible execute with previous cell

In [ ]:
Channel_config = '1ch'
if PSFtype == 'voxel' :
    Psf_config='voxel'
else :
    Psf_config='zernike'
L = psflearninglib()
L.param = io.param.combine('config_base',psftype=Psf_config,channeltype=Channel_config,sysfile=Microscope_config) 
L.param.datapath = Datapath
L.param.savename = L.param.datapath + 'psfmodel'
L.param.keyword = Keyword
L.param.subfolder = Keyword


##### **Set optional user defined parameters**

In [ ]:
# Parameters that are changed regularly
L.param.roi.max_bead_number = 22 # lower this number if out of memory 

# File and Start parameters 
# L.param.startmodel = 'D:/DataSSD/testdata/BeadsM2/psfmodel' 
# L.param.subfolder = 'Pos' # Subfolder name if each data file is saved in a individual folder, set to '' if no subfolder. Default: same as keyword

# Model parameters
L.param.option.model.const_pupilmag=False # use a constant pupil magnitude, only learn pupil phase.
L.param.option.model.symmetric_mag=False # magnitude of pupil is set symmetric
L.param.option.model.var_blur = True # use extra blurring as free parameter
L.param.option.model.blur_sigma = 1 # Units? is this pixels? or pixels/bin? start parameter for extra blurring
L.param.option.model.with_apoid=False # Use apodization
L.param.option.model.var_photon = True # allow for a variation in brightness from slice to slice for each bead separately.

# Data and FOV
# L.param.FOV.z_step = 5 # use every z_step frames. This resamples the data in z. Later we could convert this to micrometers.
# L.param.FOV.z_start=0 # raw date frames. Not intuitive: start > end. Later we could convert this to micrometers
# L.param.FOV.z_end=-0
# L.param.FOV.z_frames = 0 # give list of frames, repalces what is defined above. Allows for unequal spacing. Would be interesting to test (could give more weight to focus)
#L.param.FOV.x_center = 0 # um relative to image center
#L.param.FOV.y_center = 0
#L.param.FOV.radius = 0 # um, if radius = 0 the entire FoV is used

In [ ]:
# move to config files, remove this cell afterwards
# Microscope_config:
L.param.gain = 0.5
L.param.ccd_offset = 400
L.param.FOV.z_step = 5 
#L.param.FOV.z_step_data = .010 #in micrometer. Possible to extract from metadata?
L.param.FOV.z_start=0
L.param.FOV.z_end=-0
L.param.FOV.x_center = 0 
L.param.FOV.y_center = 0
L.param.FOV.radius = 0 

##### process those parameters, hide later

In [ ]:
# L.param.pixel_size.z = L.param.FOV.z_step * L.param.FOV.z_step_data
L.param.pixel_size.z =.05 # remove this once the code is fixed

##### Start learning

In [ ]:
images = L.load_data()
if L.param.FOV.radius >0 : # we can do some pre-processing and parameter conversion here.
    L.param.FOV.x_center=L.param.FOV.x_center+images.shape[2]/2
    L.param.FOV.y_center=L.param.FOV.y_center+images.shape[3]/2
L.getpsfclass()
dataobj = L.prep_data(images)
for k in range(0,1): # increase iteration number if necessary
    psfobj,fitter = L.learn_psf(dataobj,time=0)
resfile = L.save_result(psfobj,dataobj,fitter)

##### Show results

In [ ]:
f,p = io.h5.load(resfile) # load result file

In [ ]:
f.res.sigma

In [ ]:
showpsfvsdata(f,p,index=8)

In [ ]:
showlocalization(f,p)

##### Optional plots

In [ ]:
try:
    showpupil(f,p)
except:
    print('no pupil')

In [ ]:
try:
    showzernike(f,p)
except:
    print('no Zernike coefficients')

In [ ]:
showlearnedparam(f,p)

In [ ]:
try:
    print ('sigma:') 
    print( f.res.sigma)
except:
    print('no sigma learnt')